In [2]:
import pandas as pd
import numpy as np
import tqdm
from scipy import sparse
import torch
from torch import nn
from collections import Counter
from typing import *
import time
import logging
import altair as alt
import re
import numpy.linalg as la
import json
import Models
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from torch import nn
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import numpy.random
import spacy
import gensim
import gensim.downloader
import pickle
from collections import Counter
import LogisticRegression
from scipy.signal import find_peaks
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
alt.data_transformers.disable_max_rows
torch.manual_seed(0)
np.random.seed(0)

# 1. Data Processing

In [8]:
nlp = spacy.load('en_core_web_sm')
w2v = gensim.downloader.load('glove-wiki-gigaword-50')
df_sus_users = pd.read_csv("../Data/Network/NetworkUsers.csv", sep="\t", index_col="user_id")
def tokenize(text):
    tokens = nlp(" ".join([token.lower() for token in re.sub("\.|,|:|!|\?|-|\'", " ", text).split(" ") if token.isalpha()]))
    return [token.lemma_ for token in tokens]

def embed(tokens):
    def toVec(token):
        try:
            return w2v[token]
        except Exception:
            return w2v["unk"]
    return [toVec(token) for token in tokens]

In [ ]:
%%time
def rate(rating): # 0 is false
    for kw in ["false", "fire", "flop", "barely"]:
        if kw in rating.lower():
            return 0
    return 1

def load_data():
    df = pd.read_csv("train.csv", sep="\t", index_col="index")
    df_mis = df[df.Misinformation==1]
    df_mis = df_mis.sample(int(0.9 * len(df_mis)))
    df_true = df[df.Misinformation==0].sample(int(0.9 * len(df_mis)))
    df = pd.concat([df_true, df_mis])
    df["Statement"] = df["Statement"].apply(tokenize)
    df["Embedding"] = df["Statement"].apply(embed)
    df_val = df[~df.Source]
    df_val = df_val.sample(int(len(df_val) * 0.5))
    return {"train": df[~df.index.isin(df_val.index)], "val": df_val}
data = load_data()

# 2. LSTM

In [ ]:
def pipeline_ml(data, model: nn.Module, epochs: int=10, lr: float=1e-4):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    predictor = Models.Predictor(model, device)
    labels = torch.Tensor(data["train"].Misinformation.values)
    features = data["train"].Embedding.tolist()
    features = [torch.from_numpy(np.array(fea)) for fea in features]
    predictor.train(features, labels, epochs=epochs, lr=lr)
    return predictor

In [ ]:
lstm = Models.LSTM()
predictor = pipeline_ml(data, lstm)

In [ ]:
features = data["val"].Embedding.tolist()
features = [torch.from_numpy(np.array(fea)) for fea in features]
lstm_pred = predictor.predict(features) > 0.5

In [ ]:
f1 = f1_score(data["val"].Misinformation, lstm_pred>0.5)
acc = accuracy_score(data["val"].Misinformation, lstm_pred>0.5)
prec = precision_score(data["val"].Misinformation, lstm_pred>0.5)
recall = recall_score(data["val"].Misinformation, lstm_pred>0.5)
lstm_res = {"f1": f1, "acc": acc, "prec": prec, "recall": recall}
lstm_res

In [ ]:
lstm_pred

# 3. Logistic Regression

In [ ]:
processor = LogisticRegression.Preprocessor()
processor.buildVocabulary(data["train"])
matrix = processor.buildMatrix(data["train"])

In [ ]:
LR = LogisticRegression.LogisticRegression(processor)
LogisticRegression.train(matrix, data["train"]["Misinformation"].values, LR, lr=5e-4, epoch_num=10)

In [ ]:
lr_pred = LogisticRegression.predict(LR, data["val"]) > 0.5

In [ ]:
f1 = f1_score(data["val"].Misinformation, lr_pred>0.5)
acc = accuracy_score(data["val"].Misinformation, lr_pred>0.5)
prec = precision_score(data["val"].Misinformation, lr_pred>0.5)
recall = recall_score(data["val"].Misinformation, lr_pred>0.5)
lr_res = {"f1": f1, "acc": acc, "prec": prec, "recall": recall}
lr_res

# 4. Predict

In [ ]:
with open("GovernorPred.pkl", "wb") as f:
    f.write(pickle.dumps({"LSTM": lstm, "LR": processor}))

In [3]:
with open("GovernorPred.pkl", "rb") as f:
    models = pickle.loads(f.read())

In [4]:
predictor = Models.Predictor(models["LSTM"])
LR = LogisticRegression.LogisticRegression(models["LR"])

# 5. Inspect

In [6]:
%%time
df_tweets = pd.read_csv("../Data/Candidates/GovernorTweets.csv", sep="\t", index_col="id")
df_tweets = df_tweets[df_tweets.content!=""].dropna()
govnor_tweets = df_tweets[df_tweets.credibility!=1]
govnor_tweets = govnor_tweets.sample(int(len(govnor_tweets) * 0.25))
label = (govnor_tweets["credibility"] <= 0).astype(int)

<timed exec>:1: DtypeWarning: Columns (8,11,12) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 33.1 s, sys: 9.08 s, total: 42.2 s
Wall time: 52.3 s


In [11]:
lr_pred = LogisticRegression.predict(LR, govnor_tweets) > 0.5
f1 = f1_score(label, lr_pred>0.5)
acc = accuracy_score(label, lr_pred>0.5)
prec = precision_score(label, lr_pred>0.5)
recall = recall_score(label, lr_pred>0.5)
lr_res = {"f1": f1, "acc": acc, "prec": prec, "recall": recall}
lr_res

Build Matrix: 15304it [00:00, 52405.39it/s]


{'f1': 0.14535391785610255,
 'acc': 0.6165708311552536,
 'prec': 0.10185752194325373,
 'recall': 0.253685815963396}

In [12]:
%%time
features = govnor_tweets["content"].apply(tokenize).apply(embed).tolist()
features = [torch.from_numpy(np.array(fea)) for fea in features]
lstm_pred = predictor.predict(features) > 0.5
f1 = f1_score(label, lstm_pred>0.5)
acc = accuracy_score(label, lstm_pred>0.5)
prec = precision_score(label, lstm_pred>0.5)
recall = recall_score(label, lstm_pred>0.5)
lstm_res = {"f1": f1, "acc": acc, "prec": prec, "recall": recall}
lstm_res

  0%|          | 0/15304 [00:00<?, ?it/s]

/home/panda/anaconda3/envs/UM/lib/python3.9/site-packages/torch/nn/modules/rnn.py:761: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/cudnn/RNN.cpp:926.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


CPU times: user 1min 58s, sys: 2.31 s, total: 2min
Wall time: 2min 1s


{'f1': 0.2208536059616924,
 'acc': 0.1733533716675379,
 'prec': 0.12564821303433776,
 'recall': 0.9115404168784952}

### Tweets from incredible source not classified as misinformation

In [ ]:
# Misinformation
bad_tweets[lstm_pred].sample(10).index

In [ ]:
df_tweets.loc[[1518296660329201665, 1499864294170808332, 1516862042984095746,
            1507126074882277381, 1518231318366244867, 1505706253888143362,
            1509284007808172036, 1506148809725943808, 1501767346943414272,
            1514341548892295183]].Statement.tolist()

### Tweets from incredible source not classified as misinformation

In [ ]:
# True information
bad_tweets[~lstm_pred].sample(10).index

In [ ]:
df_tweets.loc[[1502046989483184135, 1498757468578582537, 1513529952834101255,
            1502372547442077698, 1507041373421248514, 1510022055722467333,
            1499040381463265282, 1502059266777178114, 1514618674501627904,
            1503568426861838336]].Statement.tolist()

### Tweets from credible source not classified as misinformation

In [ ]:
good_tweets[lstm_pred].sample(10).index

In [ ]:
df_tweets.loc[[1514322102303674370, 1500967005230305284, 1514267702252220418,
            1515282667251634176, 1509875045677645828, 1499209915709239296,
            1511243530202664960, 1501166881423728642, 1505590605438763011,
            1500968967040602115]].Statement.tolist()

### Tweets from credible source classified as misinformation

In [ ]:
good_tweets[~lstm_pred].sample(10).index

In [ ]:
df_tweets.loc[[1499526834341490689, 1501715649046925315, 1512255934520864770,
            1510012772712652802, 1509660956653146113, 1510813035526565889,
            1517331816398548992, 1518945587479166976, 1514944616709115904,
            1506400714544893954]].Statement.tolist()

In [ ]:
good_tweets[lstm_pred]["Statement"].apply(lambda x:len(x.split(" "))).describe()

In [ ]:
good_tweets[~lstm_pred]["Statement"].apply(lambda x:len(x.split(" "))).describe()

# 6. Use model to detect peaks

In [ ]:
def detect_peaks(governor: str, method: str="lstm", emotion: str="anger"):
    emotions = ["sadness", "joy", "love", "anger", "fear", "surprise"]
    emotion = emotions.index(emotion)
    if method == "lstm":
        features = df_gov["Statement"].apply(tokenize).apply(embed).tolist()
        features = [torch.from_numpy(np.array(fea)) for fea in features]
        lstm_pred = predictor.predict(features) > 0.5
        df_gov["pred"] = lstm_pred
    elif method == "lr":
        df_gov["pred"] = (LogisticRegression.predict(LR, df_gov) > 0.5).values
    elif method == "emotion":
        df_gov["pred"] = [res[emotion]["score"] for res in classifier(df_gov["Statement"].tolist())]
    return df_gov

In [ ]:
%%time
res = detect_peaks('Lee Zeldin', "emotion")
res

In [ ]:
# pct = res.groupby(["date"])["pred"].sum() / res.groupby(["date"])["pred"].count()
pct = res.groupby(["date"])["pred"].mean()
pct = (pct - pct.min()) / (pct.max() - pct.min()) * 100
def detect_peak(counts: pd.Series):
    prominence = 1.5 * (np.percentile(counts, 75) - np.percentile(counts, 25))
    peaks_indexes, _ = find_peaks(counts, prominence = prominence)
    return peaks_indexes
pct.iloc[detect_peak(pct)]

In [ ]:
%%time
governors = ['Stacey Abrams', 'Josh Shapiro', 'Charlie Crist', 'Katie Hobbs', 'Lee Zeldin']
peaks = dict()
for governor in governors[-1:]:
    res = detect_peaks(governor, "lr")
    pct = res.groupby(["date"])["pred"].sum() / res.groupby(["date"])["pred"].count()
    pct = (pct - pct.min()) / (pct.max() - pct.min()) * 100
    def detect_peak(counts: pd.Series):
        prominence = 1.5 * (np.percentile(counts, 75) - np.percentile(counts, 25))
        peaks_indexes, _ = find_peaks(counts, prominence = prominence)
        return peaks_indexes
    peaks[governor] = pct.iloc[detect_peak(pct)]

In [ ]:
pct.iloc[detect_peak(pct)]

# 7. Sentiment analysis

In [3]:
df_emotion = pd.read_csv("../Data/Candidates/GovernorTweets.csv", index_col="Date", sep="\t")
feature_cols = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
def load_sa_data(ratio: float=0.8, test_type: str="user"):
    if test_type == "user":
        df_sus = df_emotion[df_emotion.author_id.isin(df_sus_users.index)] 
    else:
        df_sus = df_emotion[df_emotion.credibility==0]
    df_clean = df_emotion[~df_emotion.index.isin(df_sus)]
    df_sus["label"], df_clean["label"] = 1, 0
    df_sus_train = df_sus.sample(frac=ratio)
    df_clean_train = df_clean.sample(frac=ratio)
    size = min(len(df_sus_train), len(df_clean_train))
    df_train = pd.concat([df_sus_train.sample(size), df_clean_train.sample(size)])
    df_val = pd.concat([df_sus, df_clean])
    df_val = df_val[~df_val.id.isin(df_train.id)]
    data = {"train": df_train[feature_cols + ["label"]], "val": df_val[feature_cols + ["label"]]}
    return data

In [21]:
df_emotion.Candidates.unique()

array(['Allen B. West', 'Andrew Giuliani', "Beto O'Rourke",
       'Bob Stefanowski', 'Brad Little', 'Brian Dahle', 'Brian Kemp',
       'Chad Prather', 'Charlie Crist', 'Dan Cox', 'Darren Bailey',
       'David Perdue', 'David White', 'Delilah Barrios', 'Doc Washburn',
       'Doug Mastriano', 'Greg Abbott', 'Heidi Ganahl', 'Henry McMaster',
       'J.B. Pritzker', 'Janet T. Mills', 'Jesse Sullivan', 'Jim Renacci',
       'Josh Shapiro', 'Joy Hofmeister', 'Jumaane Williams',
       'Kandiss Taylor', 'Kathy Hochul', 'Katie Hobbs', 'Kerry McQuisten',
       'Kevin Stitt', 'Kim Reynolds', 'Kristi L. Noem', 'Laura Kelly',
       'Lee Zeldin', 'Marc Thielman', 'Marco Lopez', 'Matt Brown',
       'Matt Salmon', 'Maura Healey', 'Michelle Lujan Grisham',
       'Nan Whaley', 'Ned Lamont', 'Nikki Fried', 'Paul Morgan',
       'Rebecca Dow', 'Rebecca Kleefisch', 'Richard Michael DeWine',
       'Rob Astorino', 'Ron DeSantis', 'Russell Diamond',
       'Sarah Huckabee Sanders', 'Sonia Chang-Diaz

In [9]:
import altair 

def detect_peak(counts: pd.Series, iqr: float = 1.5):
    if counts.empty:
        return list()
    prominence = iqr * (np.percentile(counts, 75) - np.percentile(counts, 25))
    peaks_indexes, _ = find_peaks(counts, prominence=prominence)
    return peaks_indexes

def plot_peak(df_counts: pd.DataFrame, field: str):
    df_counts["Counts"] = df_counts[field] / df_counts[field].max()
    df_counts = df_counts[["Counts", "Date"]]
    df_counts["Date"] = pd.to_datetime(df_counts["Date"].astype(str))
    chart = alt.Chart(df_counts).mark_line().encode(
        y=alt.Y("Counts:Q"),
        x=alt.X("Date:T"),
        tooltip=["Counts:Q", "Date:T"]
    )
    return chart

In [23]:
df_tmp = df_emotion[df_emotion["Candidates"]=="Greg Abbott"].groupby(["Date"])["anger"].mean().reset_index()
df_tmp.iloc[detect_peak(df_tmp["anger"])]

,Date,anger
5,20220306,0.270363
31,20220401,0.249340
38,20220408,0.308686
40,20220410,0.272355
45,20220415,0.301779


### Random Forest

In [9]:
%%time

# Test on suspicious users
data = load_sa_data()
rf = RandomForestClassifier()
rf.fit(data["train"][feature_cols], data["train"]["label"])
rf_pred = rf.predict(data["val"][feature_cols])
print(pd.Series(rf.feature_importances_, index=feature_cols))
Models.cal_metrics(rf_pred, data["val"]["label"])

/tmp/ipykernel_17321/3839116124.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus["label"], df_clean["label"] = 1, 0


anger       0.139366
disgust     0.152155
fear        0.144602
joy         0.144269
neutral     0.137333
sadness     0.140775
surprise    0.141500
dtype: float64
CPU times: user 52.6 s, sys: 2.89 s, total: 55.5 s
Wall time: 55.6 s


{'f1': 0.0026531852407052662,
 'acc': 0.6045953279569638,
 'prec': 0.0013296230589710648,
 'recall': 0.5820504421880118}

In [10]:
%%time

# Test on suspicious domain
data = load_sa_data(test_type="domain")
rf = RandomForestClassifier()
rf.fit(data["train"][feature_cols], data["train"]["label"])
rf_pred = rf.predict(data["val"][feature_cols])
print(pd.Series(rf.feature_importances_, index=feature_cols))
Models.cal_metrics(rf_pred, data["val"]["label"])

/tmp/ipykernel_17321/3839116124.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus["label"], df_clean["label"] = 1, 0


anger       0.143126
disgust     0.128496
fear        0.160929
joy         0.132691
neutral     0.125480
sadness     0.167031
surprise    0.142245
dtype: float64
CPU times: user 36.3 s, sys: 2.59 s, total: 38.9 s
Wall time: 38.9 s


{'f1': 0.00384708767557984,
 'acc': 0.8438570452063956,
 'prec': 0.0019289533634944632,
 'recall': 0.6859060402684564}

### Logistic Regression

In [19]:
def pipeline_sa(data, model: nn.Module, epochs: int=5, lr: float=1e-4):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    feature_cols = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
    predictor = Models.Predictor(model, device)
    labels = torch.Tensor(data["train"].label.values)
    features = [torch.from_numpy(fea) for fea in data["train"][feature_cols].values.astype(np.float32)]
    predictor.train(features, labels, epochs=epochs, lr=lr)
    return predictor

In [24]:
# Test on suspicious users
data = load_sa_data()
model = Models.LogisticRegression(7)
predictor = pipeline_sa(data=data, model=model)
df_val = data["val"].sample(frac=0.1)
lr_pred = predictor.predict(torch.from_numpy(df_val[feature_cols].values.astype(np.float32)))
Models.cal_metrics(lr_pred, df_val["label"])

  0%|          | 0/337873 [00:05<?, ?it/s]

{'f1': 0.0017919663133989404,
 'acc': 0.0008967866624441728,
 'prec': 0.0008967866624441728,
 'recall': 1.0}

In [25]:
# Test on suspicious domain
data = load_sa_data(test_type="domain")
model = Models.LogisticRegression(7)
predictor = pipeline_sa(data=data, model=model)
df_val = data["val"].sample(frac=0.1)
lr_pred = predictor.predict(torch.from_numpy(df_val[feature_cols].values.astype(np.float32)))
Models.cal_metrics(lr_pred, df_val["label"])

/tmp/ipykernel_17321/3839116124.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sus["label"], df_clean["label"] = 1, 0


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/11958 [00:00<?, ?it/s]

  0%|          | 0/11958 [00:00<?, ?it/s]

  0%|          | 0/11958 [00:00<?, ?it/s]

  0%|          | 0/11958 [00:00<?, ?it/s]

  0%|          | 0/11958 [00:00<?, ?it/s]

  0%|          | 0/338965 [00:02<?, ?it/s]

{'f1': 0.0009505278938470862,
 'acc': 0.07597539568982048,
 'prec': 0.0004754975172008833,
 'recall': 0.9675324675324676}